# Playground S3E5

The goal of the notebook is to predict the `quality` of wine using the given data. The dataset for this competition (both train and test) was generated from a deep learning model trained on the [Wine Quality dataset](https://www.kaggle.com/datasets/yasserh/wine-quality-dataset).

## Table of Contents
<a id="toc"></a>
- [1. Imports](#1)
- [2. Data Loading](#2)
- [3. EDA](#3)
- [4. Data Cleaning and Processing](#4)   
- [5. Modelling](#5)
- [6. Submission](#6)  

<a id="1"></a>
## Imports

In [2]:
%%capture
# Install extra packages
!pip install dataprep shutup lazypredict

In [18]:
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns
from dataprep.eda import create_report

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import RobustScaler
from lazypredict.Supervised import LazyClassifier
import optuna
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import make_scorer, cohen_kappa_score


import time
import warnings
warnings.filterwarnings('ignore')
import shutup
shutup.please()

<a id="2"></a>
# Load Data

This datasets is related to red variants of the Portuguese "Vinho Verde" wine. The dataset describes the amount of various chemicals present in wine and their effect on it's quality

Input variables (based on physicochemical tests):\
1 - fixed acidity\
2 - volatile acidity\
3 - citric acid\
4 - residual sugar\
5 - chlorides\
6 - free sulfur dioxide\
7 - total sulfur dioxide\
8 - density\
9 - pH\
10 - sulphates\
11 - alcohol\
Output variable (based on sensory data):\
12 - quality (score between 0 and 10)

Acknowledgements:

The original  dataset is also available from Kaggle & UCI machine learning repository, https://archive.ics.uci.edu/ml/datasets/wine+quality.

**Acknowledgements**: P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.

In [4]:
train = pd.read_csv('/kaggle/input/playground-series-s3e5/train.csv', index_col=0)
test = pd.read_csv('/kaggle/input/playground-series-s3e5/test.csv', index_col=0)
submission = pd.read_csv('/kaggle/input/playground-series-s3e5/sample_submission.csv', index_col=0)
# Load original dataset
original = pd.read_csv('/kaggle/input/wine-quality-dataset/WineQT.csv', index_col="Id")

print('Train shape:            ', train.shape)
print('Test shape:             ', test.shape)
print('Original Dataset shape: ', original.shape)

Train shape:             (2056, 12)
Test shape:              (1372, 11)
Original Dataset shape:  (1143, 12)


<a href="#toc" role="button" aria-pressed="true" > Back to Table of Contents </a>

<a id="3"></a>
## EDA

After concating `train.csv` and `WineQT.csv`:
- There are 12 columns and 3199 rows.
- 11 continous features and 1 target column
- `quality` is the target variable
- No missing values
- 125 duplicates that will be dropped
- Target value `quality` distribution is between 3-8, with large data imbalance (many 5 and 6s, very few 3, 4, and 8)
- Density and citric acid are positively correlated to fixed acidity
- Total and Free sulfer dioxide is positively correlated
- pH and fixed acidity is negatively correlated

In [5]:
# Combine train and original dataste
train = pd.concat([train, original], axis=0, ignore_index=True)
train.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,8.00,0.50,0.39,2.20,0.07,30.00,39.00,1.00,3.33,0.77,12.10,6
1,9.30,0.30,0.73,2.30,0.09,30.00,67.00,1.00,3.32,0.67,12.80,6
2,7.10,0.51,0.03,2.10,0.06,3.00,12.00,1.00,3.52,0.73,11.30,7
3,8.10,0.87,0.22,2.60,0.08,11.00,65.00,1.00,3.20,0.53,9.80,5
4,8.50,0.36,0.30,2.30,0.08,10.00,45.00,0.99,3.20,1.36,9.50,6


In [6]:
train.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,3199.00,3199.00,3199.00,3199.00,3199.00,3199.00,3199.00,3199.00,3199.00,3199.00,3199.00,3199.00
mean,8.35,0.53,0.27,2.45,0.08,16.48,48.05,1.00,3.31,0.65,10.42,5.70
std,1.72,0.18,0.19,1.07,0.03,10.12,32.93,0.00,0.15,0.15,1.05,0.84
min,4.60,0.12,0.00,0.90,0.01,1.00,6.00,0.99,2.74,0.33,8.40,3.00
25%,7.10,0.39,0.09,1.90,0.07,7.00,22.00,1.00,3.20,0.55,9.50,5.00
50%,7.90,0.52,0.25,2.20,0.08,15.00,42.00,1.00,3.31,0.62,10.10,6.00
75%,9.10,0.64,0.42,2.60,0.09,23.00,64.00,1.00,3.39,0.72,11.10,6.00
max,15.90,1.58,1.00,15.50,0.61,68.00,289.00,1.00,4.01,2.00,14.90,8.00


In [7]:
#report = create_report(train)
#report.show()

<a href="#toc" role="button" aria-pressed="true" > Back to Table of Contents </a>

<a id="4"></a>
## Data Cleaning and Processing

- Label Encode target variable
- Drop duplicates
- Train Test Split
- Over-sampling with SMOTE
- Normalization 

In [8]:
train = train.drop_duplicates()

In [9]:
X = train.drop('quality', axis=1)

# Create LabelEncoder object
le = LabelEncoder()
y = le.fit_transform(train['quality'])

X.shape, y.shape

((3074, 11), (3074,))

SMOTE (Synthetic Minority Over-sampling Technique) is an oversampling technique used to balance class distribution in binary or multi-class classification problems. The goal of oversampling is to increase the number of instances of the minority class to address class imbalance. 

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2459, 11), (615, 11), (2459,), (615,))

In [11]:
def balance_data_using_smote(X_train, y_train):
    smote = SMOTE(sampling_strategy='minority', random_state=42)
    X_smote, y_smote = smote.fit_resample(X_train, y_train)
    return X_smote, y_smote

X_train, y_train = balance_data_using_smote(X_train, y_train)
X_train.shape, y_train.shape

((3445, 11), (3445,))

In [12]:
def apply_robust_scaler(train, test):
    robust_scaler = RobustScaler()
    train_scaled = robust_scaler.fit_transform(train)
    test_scaled = robust_scaler.transform(test)
    return train_scaled, test_scaled

X_train, X_test = apply_robust_scaler(X_train, X_test)

<a href="#toc" role="button" aria-pressed="true" > Back to Table of Contents </a>

<a id="5"></a>
## Modelling


### Evaluating models

Quadratic Weighted Kappa (QWK) is a scoring metric for evaluating the agreement between two ratings. It measures the agreement between two raters, who assign categorical ratings to a number of items. Unlike simple accuracy or mean squared error, QWK takes into account the agreement beyond chance, considering both the amount and order of agreement. It ranges from -1 (representing complete disagreement) to 1 (representing complete agreement).

The metric is useful in these cases as it takes into account the possibility of different raters assigning similar scores to different conditions, while still penalizing substantial disagreements.

The quadratic weighted kappa is calculated as follows. First, an $N x N$ histogram matrix $O$ is constructed, such that $O_{i,j}$ corresponds to the number of `Ids` $i$ (actual) that received a predicted value $j$. An $N$-by-$N$ matrix of weights, $w$, is calculated based on the difference between actual and predicted values:

$$ w_{i,j} = \frac{(i-j)^2}{(N-1)^2} $$

An $N$-by-$N$ histogram matrix of expected outcomes, $E$, is calculated assuming that there is no correlation between values.  This is calculated as the outer product between the actual histogram vector of outcomes and the predicted histogram vector, normalized such that $E$ and $O$ have the same sum.

From these three matrices, the quadratic weighted kappa is calculated as: 

$$ k = 1 - \frac{\sum_{i,j} w_{i,j} O_{i,j}}{\sum_{i,j} w_{i,j} E_{i,j}} $$

In [13]:
# Define the scoring metric as quadratic_weighted_kappa
kappa_scorer = make_scorer(cohen_kappa_score, weights='quadratic')

In [14]:
#maker_scorer isn't supported for LazyPredict, so we'll make do with using default cohen_kappa_score as indicator
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=cohen_kappa_score, random_state=42) 
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
models.sort_values("cohen_kappa_score", ascending=False)

100%|██████████| 29/29 [00:14<00:00,  1.98it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,cohen_kappa_score,Time Taken
Model,,,,,,
LGBMClassifier,0.60,0.29,None,0.58,0.35,1.00
XGBClassifier,0.58,0.28,None,0.56,0.33,2.46
RandomForestClassifier,0.58,0.27,None,0.56,0.33,1.08
ExtraTreesClassifier,0.58,0.27,None,0.56,0.32,0.54
SVC,0.56,0.29,None,0.55,0.31,0.60
BaggingClassifier,0.54,0.26,None,0.53,0.27,0.30
QuadraticDiscriminantAnalysis,0.52,0.27,None,0.52,0.26,0.03
LogisticRegression,0.46,0.36,None,0.48,0.23,0.14
LinearDiscriminantAnalysis,0.45,0.36,None,0.48,0.22,0.09


### Model Tuning

The performance of a machine learning model is highly dependent on the choice of hyperparameters. Hyperparameters determine the architecture and behavior of a model and can greatly affect the model's accuracy, precision, recall, and overall performance.

In [22]:
def tune_extra_trees_classifier(X_train, y_train, n_splits=5, trials=100):
    def objective(trial):
        params = {
        "n_estimators": trial.suggest_int('n_estimators', 50, 500),
        "max_depth": trial.suggest_int('max_depth', 2, 30),
        "min_samples_split": trial.suggest_int('min_samples_split', 2, 20),
        "min_samples_leaf": trial.suggest_int('min_samples_leaf', 1, 20),
        "max_features": trial.suggest_uniform('max_features', 0.1, 1.0)
        }
        model = ExtraTreesClassifier(**params, random_state=42)
        cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
        kappa_scores = -1 * cross_val_score(model, X, y, cv=cv, scoring=kappa_scorer)
        return np.mean(kappa_scores)
    
    study = optuna.create_study()
    study.optimize(objective, n_trials=trials)
    best_params = study.best_params
    best_score = study.best_value
    return best_params, best_score

def tune_lgbm(X_train, y_train, n_splits=5, trials=100):
    def objective(trial):
        params = {
            "objective" : 'multiclass',
            "boosting_type": trial.suggest_categorical("boosting_type", ["gbdt", "dart", "goss"]),
            "num_leaves": trial.suggest_int("num_leaves", 10, 100),
            "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1.0),
            "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
            "subsample": trial.suggest_uniform("subsample", 0.1, 1.0),
            "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.1, 1.0),
        }
        model = lgb.LGBMClassifier(**params)
        cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
        kappa_scores = -1 * cross_val_score(model, X, y, cv=cv, scoring=kappa_scorer)
        return np.mean(kappa_scores)
    
    study = optuna.create_study()
    study.optimize(objective, n_trials=trials)
    best_params = study.best_params
    best_score = study.best_value
    return best_params, best_score

def tune_xgb_classifier(X, y, n_splits=5, trials=100):
    def objective(trial):
        xgb_params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 20),
            'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.3),
            'subsample': trial.suggest_uniform('subsample', 0.3, 1.0),
            'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3, 1.0),
            'reg_alpha': trial.suggest_uniform('reg_alpha', 0.0, 1.0),
            'reg_lambda': trial.suggest_uniform('reg_lambda', 0.0, 1.0),
            'objective': 'multi:softmax',
            'num_class': len(np.unique(y))
        }
        model = xgb.XGBClassifier(**xgb_params)
        cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
        kappa_scorer = make_scorer(cohen_kappa_score, weights='quadratic')
        kappa_scores = -1 * cross_val_score(model, X, y, cv=cv, scoring=kappa_scorer)
        return np.mean(kappa_scores)

    study = optuna.create_study()
    study.optimize(objective, n_trials=trials)
    best_params = study.best_params
    best_value = study.best_value
    return best_params, best_value

def tune_rf_classifier(X, y, n_splits=5, trials=100):
    def objective(trial):
        rf_params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 1, 30),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
            'max_features': trial.suggest_uniform('max_features', 0.1, 1.0),
        }
        model = RandomForestClassifier(**rf_params)
        cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
        kappa_scorer = make_scorer(cohen_kappa_score, weights='quadratic')
        kappa_scores = -1 * cross_val_score(model, X, y, cv=cv, scoring=kappa_scorer)
        return np.mean(kappa_scores)

    study = optuna.create_study()
    study.optimize(objective, n_trials=trials)
    best_params = study.best_params
    best_value = study.best_value
    return best_params, best_value

In [3]:
# Tune hyper-parameters using optuna
#tune_extra_trees_classifier(X_train, y_train, n_splits=5, trials=100)
#tune_lgbm(X_train, y_train, n_splits=5, trials=1)
#tune_xgb_classifier(X_train, y_train, n_splits=5, trials=100)
#tune_rf_classifier(X_train, y_train, n_splits=5, trials=100)

In [2]:
# If params not defined, used hard-coded values from previous hyper-parameter tuning
if 'trees_classifier_params' not in locals():
    trees_classifier_params = {
        'n_estimators': 483,
        'max_depth': 20,
        'min_samples_split': 9,
        'min_samples_leaf': 1,
        'max_features': 0.9760192595957563
    }

if 'lgbm_params' not in locals():
    lgbm_params = {
        'boosting_type': 'gbdt',
        'num_leaves': 90,
        'learning_rate': 1.2846185076835063e-05,
        'n_estimators': 568,
        'min_child_weight': 4,
        'subsample': 0.2078432609415698,
        'colsample_bytree': 0.9785173128446786
    }

if 'xgb_params' not in locals():
    xgb_params = {
        'n_estimators': 515,
        'max_depth': 12,
        'learning_rate': 0.04876788920845744,
        'subsample': 0.5820311109870313,
        'colsample_bytree': 0.34985037004316355,
        'reg_alpha': 0.34472042964865496,
        'reg_lambda': 0.11032246179781986
    }

if 'rf_params' not in locals():  
    rf_params = {
        'n_estimators': 968,
        'max_depth': 25,
        'min_samples_split': 14,
        'min_samples_leaf': 1,
        'max_features': 0.7048284037565753
    }

In [ ]:
xgb_params = {          
    'subsample'       : 0.1,
    'reg_lambda'      : 50,
    'min_child_weight': 1,
    'max_depth'       : 6,
    'learning_rate'   : 0.05,
    'colsample_bytree': 0.4,
    'objective'       : 'multi:softmax',
    'eval_metric'     : 'mlogloss',
}

<a href="#toc" role="button" aria-pressed="true" > Back to Table of Contents </a>

<a id="6"></a>
## Submssion

In [ ]:
X, y = balance_data_using_smote(X, y)
X, test = apply_robust_scaler(X, test)

In [ ]:
model = xgb.XGBClassifier(**xgb_params)
model.fit(X, y)
y_pred = model.predict(test)
# Reverse the encoding
predictions = le.inverse_transform(y_pred)

In [ ]:
assert len(submission) == len(predictions), "Length of submission DataFrame and predictions array are unequal"

submission.quality = predictions
submission.head()

In [ ]:
submission.to_csv('submission.csv')